# DUYGU ANALİZ UYGULAMASI

## Veri Ön İşleme

In [1]:
#Veri ön işlemede kullanacağımız spacy kütüphanesi'ni kodumuza dahil ediyoruz.
import spacy
from spacy import displacy
import en_core_web_sm
nlp = en_core_web_sm.load()

### Kelime Köklerini Bulma (Lemmatization)


In [2]:
#ingilizce kelimelerin köklerini bulalım
ornek = nlp("teach teaches teaching taught")
for lem in ornek:
    print("kelime:" + lem.text + ", kök:" + lem.lemma_)

kelime:teach, kök:teach
kelime:teaches, kök:teach
kelime:teaching, kök:teach
kelime:taught, kök:teach


### Cümlenin Ögelerine Ayrılması ve Zamirlerin Veriden Çıkarılması

In [3]:
#ingilizce bir cümleyi ögelerine ayıralım
#zamirler tespit edildikten sonra veri ön işleme fonksiyonuyla veri setinden çıkarılacak
ornek = nlp('A bad workman always blames his tools.')
for token in ornek:
    print(token.text + "\t" + token.pos_)

A	DET
bad	ADJ
workman	NOUN
always	ADV
blames	VERB
his	DET
tools	NOUN
.	PUNCT


In [4]:
displacy.render(ornek, style = 'dep' , options={"distance": 140})

### Etkisiz Kelimeler (Stop-words)

In [5]:
#Spacy Kütüphanesi'nde bulunan İngilizce stop-word'leri kodumuzun içine alalım.
#Stop-word'leri veri ön işleme fonksiyonunu yazarken kullanacağız.
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
print(stopwords)

['nine', 'since', 'everyone', 'another', 'hundred', 'at', 'via', 'using', 'thereupon', 'you', 'various', 'along', 'out', 'hereupon', 'that', 'almost', 'only', 'thereby', 'perhaps', 'same', '’d', 'when', 'quite', 'eight', 'yourselves', 'around', 'before', 'whatever', 'onto', 'full', 'upon', 'namely', 'other', 'none', 'two', 'whole', 'more', '’m', 'if', 'hence', 'was', 'nevertheless', 'mine', 'third', 'are', 'beforehand', 'though', 'her', 'last', 'myself', 'due', 'whereas', 'whenever', 'must', 'bottom', 'sixty', 'whose', 'except', 'which', 'even', 'nobody', 'ca', 'again', 'noone', 'everywhere', 'amongst', 'over', 'beside', 'rather', 'my', 'above', 'me', 'could', 'of', 'had', 'first', 'twelve', 'never', 'give', 'himself', 'wherein', 'several', 'amount', 'become', 'has', 'however', '‘ll', 'each', 'becoming', 'for', 'your', 'would', 'both', 'once', 'sometime', 'no', 'during', 'should', 'anyone', '‘d', 'whom', 'part', 'put', 'can', 'top', 'front', '’ll', 'she', 'indeed', 'latterly', 'side', 

In [6]:
#Spacy kütüphanesinde bulunan ingilizce stop-words sayısı
len(stopwords)

326

### Veriden Noktalama İşaretlerinin (Punctuation) Çıkarılması

In [7]:
#Noktalama işaretlerini kodun içine alalım. 
#Bu işaretleri veri ön işleme fonksiyonunu yazarken kullanacağız.
import string
noktalama = string.punctuation
noktalama

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Cümleleri Daha Küçük Anlamlı Birimlere Ayırma (Tokenization)

In [8]:
#Veriyi token'lerine ayıran, veriden zamirleri çıkaran, kelimelerin kökünü bulan, tüm karakterleri küçük hale getiren,
#veriden stop word'leri ve noktalama işaretlerini ayıklayan veri ön işleme fonksiyonunu tanımlayalım.

def veri_on_isleme(cumle):
    doc = nlp(cumle)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in noktalama:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [9]:
veri_on_isleme("Hello there! What's up dude,   I like your style!!")

['hello', 'dude', 'like', 'style']

### İsimlerin Üst Kavramını Bulma (Named Entity Recognition)

In [10]:
#Örnek olarak aşağıda verilen haber metninin içinde geçen isimlerin üst kavramlarını bulalım.
ornek = nlp("Russia and Germany reported record daily Covid-19 deaths on Friday, and October was Russia's deadliest month in a decade. Stockholm's intensive care units hit 99% capacity as Sweden proposed a spring 'pandemic law' to potentially force closures of certain public spaces. And France said its lockdown would not be eased as planned on Tuesday after daily case numbers rose on Thursday compared with last week.Germany tallied 598 fatalities in a span of 24 hours, according to the country's center for disease control, the Robert Koch Institute. 'Germany clearly is very worried and you can see their numbers are absolutely going in the wrong direction', Linda Bauld, professor of public health at the University of Edinburgh, told CNN. December 11, 2020 ")
print(ornek)

Russia and Germany reported record daily Covid-19 deaths on Friday, and October was Russia's deadliest month in a decade. Stockholm's intensive care units hit 99% capacity as Sweden proposed a spring 'pandemic law' to potentially force closures of certain public spaces. And France said its lockdown would not be eased as planned on Tuesday after daily case numbers rose on Thursday compared with last week.Germany tallied 598 fatalities in a span of 24 hours, according to the country's center for disease control, the Robert Koch Institute. 'Germany clearly is very worried and you can see their numbers are absolutely going in the wrong direction', Linda Bauld, professor of public health at the University of Edinburgh, told CNN. December 11, 2020 


In [11]:
displacy.render(ornek, style = 'ent')

In [12]:
#Veri setimizi kodumuza import edelim
file = open('data/birlestirilmis.txt', 'r') 
string = file.read() 

In [13]:
#Veri setimizde geçen tüm isimleri üst kavram etiketleri ile değiştirelim
doc = nlp(string)
newString = doc.text
counter = 0
for ent in reversed(doc.ents):
    newString = newString[:ent.start_char] + ent.label_ + newString[ent.end_char:]
    counter += 1
    if counter>500 and counter<530:
        print("iliski('" +ent.text+ "','üst kavramı nedir?','" + ent.label_+ "')" )

iliski('one','üst kavramı nedir?','CARDINAL')
iliski('Alexander Nevsky','üst kavramı nedir?','PERSON')
iliski('2 hours','üst kavramı nedir?','TIME')
iliski('Tom Hanks','üst kavramı nedir?','PERSON')
iliski('10','üst kavramı nedir?','CARDINAL')
iliski('9','üst kavramı nedir?','CARDINAL')
iliski('one','üst kavramı nedir?','CARDINAL')
iliski('Marine','üst kavramı nedir?','NORP')
iliski('Sam Shepard's','üst kavramı nedir?','PERSON')
iliski('Anne Heche','üst kavramı nedir?','PERSON')
iliski('one','üst kavramı nedir?','CARDINAL')
iliski('another 2 hours','üst kavramı nedir?','TIME')
iliski('Cancan','üst kavramı nedir?','NORP')
iliski('French','üst kavramı nedir?','NORP')
iliski('one','üst kavramı nedir?','CARDINAL')
iliski('Ed Helms','üst kavramı nedir?','PERSON')
iliski('Jonah Hill','üst kavramı nedir?','PERSON')
iliski('Morgan Freeman','üst kavramı nedir?','ORG')
iliski('Carrell','üst kavramı nedir?','ORG')
iliski('ENTIRE','üst kavramı nedir?','ORG')
iliski('Canada','üst kavramı nedir?','G

### TF-IDF Metin Temsil Yöntemi

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [15]:
tfidf = TfidfVectorizer(tokenizer = veri_on_isleme)

### Veri Seti Üzerinde Diğer Ön İşlemler

In [16]:
#String formatındaki veriyi pandas dataframe formatına çevirme
import io
from io import StringIO
import pandas as pd
temporary = io.StringIO(newString)
data = pd.read_csv(temporary, sep="\t", header = None)
data[:5]

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the DATE bank holiday off PE...,1
4,The selection on the menu was great and so wer...,1


In [17]:
#Tabloya başlıklar ekleme
columns_name = ['Yorum', 'Duygu']
data.columns = columns_name
data[:5]

,Yorum,Duygu
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the DATE bank holiday off PE...,1
4,The selection on the menu was great and so wer...,1


In [18]:
data.shape

(3000, 2)

In [19]:
#Duygu durumuna göre (0: negatif, 1: pozitif) yorum sayıları
data['Duygu'].value_counts()

1    1500
0    1500
Name: Duygu, dtype: int64

In [20]:
#Veri seti içinde null değer olup olmadığının kontrolünü yapalım
data.isnull().sum()

Yorum    0
Duygu    0
dtype: int64

## Metin Sınıflandırma

In [21]:
#Kullanıcalacak sınıflandırma algoritması için gerekli kütiphaneleri import edelim
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

classifier = LinearSVC()

In [22]:
#Veriyi yorum verisi ve duygu verisi olmak üzere ikiye bölelim
X = data['Yorum']
y = data['Duygu']

In [23]:
#Eğitim ve test kümesini birbirinden ayıralım
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 36)
X_train.shape, X_test.shape

((2400,), (600,))

In [24]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [25]:
clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function veri_on_isleme at 0x000002B4B0336B80>)),
                ('clf', LinearSVC())])

In [26]:
y_pred = clf.predict(X_test)

In [27]:
#Sınıflandırma raporunun oluşturulması
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78       295
           1       0.78      0.80      0.79       305

    accuracy                           0.79       600
   macro avg       0.79      0.79      0.79       600
weighted avg       0.79      0.79      0.79       600



In [28]:
#Hata matrisinin oluşturulması
confusion_matrix(y_test, y_pred)

array([[227,  68],
       [ 60, 245]], dtype=int64)

In [29]:
#Elle girilen yeni cümlelerle modelin test edilmesi için fonksiyon oluşturalım
def duygu_analizi(test_cumlesi):
    if clf.predict([test_cumlesi])[0]==1:
        print("duygu : pozitif (1)")
    else:
        print("duygu : negatif (0)")

In [30]:
duygu_analizi("I like adventure movies")

duygu : pozitif (1)


In [31]:
duygu_analizi("I believe my salary is too low for what I'm doing.")

duygu : negatif (0)


In [32]:
duygu_analizi('You are amazing.')

duygu : pozitif (1)


In [33]:
duygu_analizi('Such a waste of my time...')

duygu : negatif (0)


In [34]:
duygu_analizi("This is fabulous work, friend.")

duygu : pozitif (1)
